In [10]:
import sys
sys.path.append("/home/ms10596/PycharmProjects/match")
from utils.one_stop_english import load_advanced_elementary
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from statistics import mean
import numpy as np


In [11]:
advanced, elementary = load_advanced_elementary()

In [12]:
print("mean len of adv:", mean([len(i) for i in advanced]))
print("mean len of elem:", mean([len(i) for i in elementary]))

mean len of adv: 126.8915050784857
mean len of elem: 111.70452446906741


In [13]:
print(advanced[50])
print(elementary[50])

china is the biggest coal importer, and indonesia the biggest exporter, having temporarily overtaken australia.
china is the biggest coal importer, and indonesia is the biggest coal exporter.


In [14]:
source_sequences = tokenizer.texts_to_sequences(advanced)
print(np.shape(source_sequences))
print(source_sequences[0])
print(advanced[0])

(2166,)
[1, 3290, 475, 197, 22, 1089, 9, 34, 948, 4, 28, 5, 118, 233, 715, 412, 1430, 517, 17, 1, 164, 302, 1876, 1090, 41, 64, 1297, 1, 96, 3, 41, 277, 829, 9, 884, 1877, 1, 3291, 3, 1091, 1878, 2, 71, 204, 885, 606]
the seattle-based company has applied for its brand to be a top-level domain name (currently .com), but the south american governments argue this would prevent the use of this internet address for environmental protection, the promotion of indigenous rights and other public interest uses.


In [15]:
target_sequences = tokenizer.texts_to_sequences(elementary)
print(np.shape(target_sequences))
print(target_sequences[0])
print(elementary[0])

(2166,)
[887, 22, 350, 9, 34, 197, 412, 4, 28, 5, 118, 233, 715, 412, 1430, 517, 17, 1, 164, 302, 1876, 212, 41, 64, 652, 1, 96, 3, 41, 277, 829, 9, 884, 1877, 1091, 1878, 2, 71, 204, 885, 606]
amazon has asked for its company name to be a top-level domain name (currently .com), but the south american governments say this would stop the use of this internet address for environmental protection, indigenous rights and other public interest uses.


In [16]:
print("mean len of adv:", max([len(i) for i in source_sequences]))
print("mean len of elem:", max([len(i) for i in target_sequences]))

mean len of adv: 60
mean len of elem: 52


In [17]:
from tensorflow.keras.utils import to_categorical
padded_target_sequences = pad_sequences(target_sequences, maxlen=60, padding='post', truncating='post')
padded_source_sequences = pad_sequences(source_sequences, maxlen=60, padding='post', truncating='post')
padded_target_sequences = to_categorical(padded_target_sequences)
print(np.shape(padded_target_sequences))
print(np.shape(padded_source_sequences))

(2166, 60, 5761)
(2166, 60)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, TimeDistributed, RepeatVector, Dense
from tensorflow.python.keras.layers.embeddings import Embedding
from tensorflow import keras
es = keras.callbacks.EarlyStopping(monitor='val_loss',patience=10, verbose=0, mode='auto')

n_units = 256
model = Sequential()
model.add(Embedding(5761, n_units, input_length=60, mask_zero=True))
# model.add(LSTM(n_units))
# model.add(RepeatVector(tar_timesteps))
model.add(LSTM(n_units, return_sequences=True))
model.add(TimeDistributed(Dense(5761, activation='softmax')))
model.compile(optimizer='adam', loss='categorical_crossentropy')
model.fit(padded_source_sequences, padded_target_sequences, epochs=500, batch_size=64, verbose=2, validation_split=0.15,callbacks=[es])

/home/ms10596/anaconda3/envs/tf/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 1841 samples, validate on 325 samples
Epoch 1/500


In [ ]:
id_to_word = {i:word for word,i in tokenizer.word_index.items()}
id_to_word[0] = "<space>"

In [ ]:
for i in model.predict(padded_source_sequences[50:51])[0]:
#     print(np.shape(i))
    val = np.argmax(i)
    if val < 5761:
        print(id_to_word[val])

In [27]:
model.save('sha2y.h5')